In [1]:
from numpy import expand_dims
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np
import json
import cv2
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

def extract_face(filename, required_size=(224, 224)):
    print("[INFO] extracting only face from the image...")
    pixels = plt.imread(filename)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
 
print("[INFO] loading image...")
pixels = extract_face('sharon_stone1.jpg')
pixels = pixels.astype('float32')

print("[INFO] expand dimensions...")
samples = expand_dims(pixels, axis=0)

print("[INFO] preprocess image...")
samples = preprocess_input(samples, version=2)

print("[INFO] loading model...")
model = VGGFace(model='resnet50')

print("[INFO] making prediction...")
yhat = model.predict(samples)

print("[INFO] decoding predictions...")
results = decode_predictions(yhat)
for result in results[0]:
    print('%s: %.3f%%' % (str(result[0].encode('utf-8').decode('utf-8')), result[1]*100))

[INFO] loading image...
[INFO] extracting only face from the image...
[INFO] expand dimensions...
[INFO] preprocess image...
[INFO] loading model...
[INFO] making prediction...
[INFO] decoding predictions...
b' Sharon_Stone': 99.574%
b' Noelle_Reno': 0.080%
b' Anita_Lipnicka': 0.027%
b' Elisabeth_R\xc3\xb6hm': 0.027%
b' Emma_Atkins': 0.019%


In [2]:

with open('labels.json',"r+") as f:
    l = f.read()

data = json.loads(l)

In [3]:
from tensorflow.keras.applications.resnet50 import preprocess_input

def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    image = np.expand_dims(image, axis=0)
    return image

def clip_eps(tensor, eps):
    return tf.clip_by_value(tensor, clip_value_min=-eps, clip_value_max=eps)


def generate_adversaries(model, baseImage, delta, classIdx, steps=50):
    for step in range(0, steps):

        with tf.GradientTape() as tape:
        
            tape.watch(delta)

            adversary = preprocess_input(baseImage + delta)
            predictions = model(adversary, training=False)
            loss = -sccLoss(tf.convert_to_tensor([classIdx]), predictions)
            
            if step % 5 == 0:
                print("[INFO] step: {}, loss: {}...".format(step,
                    loss.numpy()))
        
            gradients = tape.gradient(loss, delta)
            optimizer.apply_gradients([(gradients, delta)])
            delta = delta.assign_add(clip_eps(delta, eps=EPS))
    return delta

EPS = 2 / 255
LR = 0.1

optimizer = Adam(learning_rate=LR)
sccLoss = SparseCategoricalCrossentropy()

print("[INFO] loading image...")
pixels = extract_face('sharon_stone1.jpg')
pixels = pixels.astype('float32')

print("[INFO] preprocessing image...")
image = preprocess_image(pixels)

print("[INFO] init consta and variablese...")
baseImage = tf.constant(image, dtype=tf.float32)
delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)

print("[INFO] generating adversarials watching delta...")
deltaUpdated = generate_adversaries(model, baseImage, delta, int(data[' Sharon_Stone']))

print("[INFO] creating adversarial example...")
adverImage = (baseImage + deltaUpdated).numpy().squeeze()
adverImage = np.clip(adverImage, 0, 255).astype("uint8")
adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)

print("[INFO] saving the adversarial example...")
cv2.imwrite("adverImage.png", adverImage)



[INFO] loading image...
[INFO] extracting only face from the image...
[INFO] preprocessing image...
[INFO] init consta and variablese...
[INFO] generating adversarials watching delta...
[INFO] step: 0, loss: -0.006777158007025719...
[INFO] step: 5, loss: -0.10115277767181396...
[INFO] step: 10, loss: -2.4546256065368652...
[INFO] step: 15, loss: -7.9793829917907715...
[INFO] step: 20, loss: -12.94911003112793...
[INFO] step: 25, loss: -16.84220314025879...
[INFO] step: 30, loss: -19.839214324951172...
[INFO] step: 35, loss: -22.07464599609375...
[INFO] step: 40, loss: -23.84256362915039...
[INFO] step: 45, loss: -25.312740325927734...
[INFO] creating adversarial example...
[INFO] saving the adversarial example...


True

In [4]:
print("[INFO] reading adversarial image example...")
image = cv2.imread("adverImage.png")

print("[INFO] preprocessing adversarial image example...")
image = preprocess_image(image)

preprocessedImage = preprocess_input(image)

print("[INFO] making predictions with adversarial image example...")
predictions = model.predict(preprocessedImage)

print("[INFO] decoding predictions with adversarial image example...")
predictions = decode_predictions(predictions)

for result in predictions[0]:
    print('%s: %.3f%%' % (str(result[0].encode('utf-8').decode('utf-8')), result[1]*100))    

[INFO] reading adversarial image example...
[INFO] preprocessing adversarial image example...
[INFO] making predictions with adversarial image example...
[INFO] decoding predictions with adversarial image example...
b' Bobbi_Sue_Luther': 99.998%
b' Sam_Pinto': 0.000%
b' Maura_Rivera': 0.000%
b' Georgina_Verbaan': 0.000%
b' Agnieszka_Maciag': 0.000%
